# Semanttinen ydin OpenBnB MCP-palvelimen integroinnilla

Tämä muistikirja näyttää, kuinka käyttää Semanttista ydintä todellisen OpenBnB MCP-palvelimen kanssa etsimään oikeita Airbnb-majoituksia MCPStdioPluginin avulla. LLM-yhteyden osalta käytetään Azure AI Foundrya. Ympäristömuuttujien asettamiseksi voit seurata [Asennusopasta](/00-course-setup/README.md)


## Tuo tarvittavat paketit


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP-liitännäisen yhteyden luominen

Yhdistämme [OpenBnB MCP -palvelimeen](https://github.com/openbnb-org/mcp-server-airbnb) käyttämällä MCPStdioPlugin-liitännäistä. Tämä palvelin tarjoaa Airbnb-hakutoiminnallisuuden @openbnb/mcp-server-airbnb-paketin kautta.


## Asiakkaan luominen

Tässä esimerkissä käytämme Azure AI Foundrya LLM-pääsyyn. Varmista, että ympäristömuuttujat on asetettu oikein.


## Ympäristön konfigurointi

Määritä Azure OpenAI -asetukset. Varmista, että seuraavat ympäristömuuttujat on asetettu:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP -integraation ymmärtäminen

Tämä muistikirja yhdistyy **todelliseen OpenBnB MCP -palvelimeen**, joka tarjoaa oikean Airbnb-hakutoiminnallisuuden.

### Miten se toimii:

1. **MCPStdioPlugin**: Käyttää standardia syöttö-/tulostuskommunikaatiota MCP-palvelimen kanssa
2. **Todellinen NPM-paketti**: Lataa ja suorittaa `@openbnb/mcp-server-airbnb` npx:n avulla
3. **Reaaliaikaiset tiedot**: Palauttaa oikeita Airbnb-kohdetietoja heidän APIensa kautta
4. **Toimintojen tunnistus**: Agentti tunnistaa automaattisesti käytettävissä olevat toiminnot MCP-palvelimelta

### Käytettävissä olevat toiminnot:

OpenBnB MCP -palvelin tarjoaa yleensä seuraavat toiminnot:
- **search_listings** - Etsi Airbnb-kohteita sijainnin ja kriteerien perusteella
- **get_listing_details** - Hanki yksityiskohtaisia tietoja tietyistä kohteista
- **check_availability** - Tarkista saatavuus tietyille päivämäärille
- **get_reviews** - Hae arvosteluja kohteista
- **get_host_info** - Hanki tietoja kohteiden isännistä

### Esivaatimukset:

- **Node.js** asennettuna järjestelmääsi
- **Internet-yhteys** MCP-palvelinpaketin lataamista varten
- **NPX** käytettävissä (sisältyy Node.js:ään)

### Yhteyden testaaminen:

Voit testata MCP-palvelinta manuaalisesti suorittamalla:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Tämä lataa ja käynnistää OpenBnB MCP -palvelimen, johon Semantic Kernel sitten yhdistyy saadakseen oikeita Airbnb-tietoja.


## Agentin käynnistäminen OpenBnB MCP -palvelimen kanssa

Nyt käynnistämme tekoälyagentin, joka yhdistyy OpenBnB MCP -palvelimeen etsiäkseen oikeita Airbnb-majoituksia Tukholmasta kahdelle aikuiselle ja yhdelle lapselle. Voit vapaasti muokata `user_inputs`-listaa muuttaaksesi hakukriteerejä.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Yhteenveto
Onnittelut! Olet onnistuneesti rakentanut tekoälyagentin, joka integroituu todelliseen majoitushakuun käyttämällä Model Context Protocolia (MCP):

## Käytetyt teknologiat:
- Semantic Kernel - Älykkäiden agenttien rakentamiseen Azure OpenAI:n avulla
- Azure AI Foundry - LLM-ominaisuuksiin ja keskustelujen täydentämiseen
- MCP (Model Context Protocol) - Standardoituun työkalujen integrointiin
- OpenBnB MCP Server - Todelliseen Airbnb-hakutoiminnallisuuteen
- Node.js/NPX - Ulkoisen MCP-palvelimen suorittamiseen

## Mitä olet oppinut:
- MCP-integraatio: Semantic Kernel -agenttien yhdistäminen ulkoisiin MCP-palvelimiin
- Reaaliaikainen tiedonsaanti: Todellisten Airbnb-kohteiden haku live-rajapintojen kautta
- Protokollaviestintä: stdio-viestinnän käyttö agentin ja MCP-palvelimen välillä
- Toimintojen löytäminen: MCP-palvelimien tarjoamien toimintojen automaattinen tunnistaminen
- Vastausten suoratoisto: Funktiokutsujen tallentaminen ja lokittaminen reaaliajassa
- HTML-renderöinti: Agentin vastausten muotoilu tyylitellyillä taulukoilla ja interaktiivisilla näkymillä

## Seuraavat askeleet:
- Integroi lisää MCP-palvelimia (sää, lennot, ravintolat)
- Rakenna monen agentin järjestelmä, joka yhdistää MCP- ja A2A-protokollat
- Luo omia MCP-palvelimia omille tietolähteillesi
- Toteuta pysyvä keskustelumuisti istuntojen välillä
- Ota agentti käyttöön Azure Functions -ympäristössä MCP-palvelimen orkestroinnilla
- Lisää käyttäjätunnistus ja varausominaisuudet



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskääntämistä. Emme ole vastuussa tämän käännöksen käytöstä aiheutuvista väärinkäsityksistä tai virhetulkinnoista.
